Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## Define Strategy

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

In [2]:
class SimpleStrategy(Strategy):
    def __init__(self, asset, quantity):
        super().__init__()
        self.asset = asset
        self.quantity = quantity

    def log_all(self, orders, data, ctx, utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(utc)
            self.log_ohlcv(data, self.asset.symbol, ctx.exchange.id)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        new_orders = []
        quantity = .05
        price = data.get('close', self.asset.symbol, ctx.exchange.id)
        current_time = data.get('utc')
        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.record.portfolio.balance, ctx.exchange, self.asset, quantity, price, current_time)
            new_orders.append(order)
        elif ctx.record.balance.get(self.asset.base)[BalanceType.FREE] > quantity:
            order = order_manager.build_market_sell_order(
                ctx.record.portfolio.balance, ctx.exchange, self.asset, quantity, current_time)
            new_orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)
        self.log_all(new_orders, data, ctx, current_time)
        return {
            'new_orders': new_orders,
            'cancel_ids': cancel_ids
        }

## Backtest

### Backtest From Local CSV Files

In [ ]:
# Download some data from Binance
exchange_id = ex_cfg.BINANCE
timeframe = Timeframe.THIRTY_MIN
start = datetime.datetime(year=2018, month=1, day=1, hour=0)
end = datetime.datetime(year=2018, month=1, day=5, hour=0)
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.BTC, coins.USDT)]
data_exchange = load_exchange(exchange_id)
ohlcv_feed.download_ohlcv([data_exchange], assets, timeframe, start, end)

In [ ]:
# Setup the Feed Based Paper Exchange
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=cash_currency,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=timeframe
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_balance=deepcopy(balance),
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = OHLCVFileFeed(
    exchange_ids=[exchange_id],
    assets=assets,
    timeframe=timeframe,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(deepcopy(balance), feed, exchange_id)
strategy = SimpleStrategy(assets[0], quantity=.05)
experiment_name = 'default_backtest'

In [ ]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

In [ ]:
record

### Backtest from External Exchange Data

In [ ]:
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=cash_currency,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, ex_cfg.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_balance=balance,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow()-datetime.timedelta(days=1),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

In [ ]:
experiment_name = 'default_backtest_exchange'
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

## Simulate Orders

In [3]:
cash = 1.0
cash_currency = coins.BTC
balance = Balance(
    cash_currency=coins.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, ex_cfg.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_balance=balance,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)

assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy(assets[0], quantity=.05)

Downloading: ETH/BTC
Downloaded rows: 0
Downloading: LTC/BTC
Downloaded rows: 0
Downloading: BTC/USDT
Downloaded rows: 0


In [4]:
experiment_name = 'default_simulate'
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-30 04:52:41.884227
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-30 04:52:41.884227
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-30 04:52:41.884227
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-30 04:52:41.884227
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data after prior poll: 2018-01-30 04:52:41.884227
Downloading: ETH/BTC
Downloaded rows: 1
Downloading: LTC/BTC
Downloaded rows: 1
Downloading: BTC/USDT
Downloaded rows: 1
No data a

KeyboardInterrupt: 

## Live Trade

In [ ]:
exchange = load_exchange(ex_cfg.BINANCE)
cash = exchange.fetch_balance().get(coins.BTC)[BalanceType.FREE]
cash_currency = coins.BTC
print(coins.BTC, "cash", cash)

In [ ]:
balance = Balance(
    cash_currency=coins.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    cash_currency=cash_currency,
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(coins.ETH, coins.BTC), Asset(coins.LTC, coins.BTC)]
feed = OHLCVExchangeFeed(
    exchanges=[exchange],
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=None,
    end=None
)
experiment_name = 'default_live'

In [ ]:
runner.live(experiment_name, exchange, balance, portfolio, feed, strategy)